In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
DATA_PATH = './'

In [3]:
df = pd.read_csv(DATA_PATH + 'data/iris_extended.csv')
df = df[["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]]
df_ = df.drop(columns=["species"])


In [4]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.16,3.41,1.64,0.26,setosa
1,5.48,4.05,1.53,0.37,setosa
2,5.10,2.80,1.47,0.38,setosa
3,4.64,3.44,1.53,0.17,setosa
4,4.85,2.87,1.23,0.26,setosa


In [5]:
def Gaus_neuron(df, n, step, s):

    neurons_list = list()
    x_axis_list = list()
    t = 0

    for col in df.columns:

        vol = df[col].values
        min_ = np.min(vol)
        max_ = np.max(vol)
        x_axis = np.arange(min_, max_, step)
        x_axis[-1] = max_
        x_axis_list.append(np.round(x_axis, 10))
        neurons = np.zeros((n, len(x_axis)))

        for i in range(n):

            loc = (max_ - min_) * (i /(n-1)) + min_
            neurons[i] = norm.pdf(x_axis, loc, s[t])
            neurons[i] = neurons[i] / np.max(neurons[i])

        neurons_list.append(neurons)
        t += 1

    return neurons_list, x_axis_list

In [6]:
def Lat_Spike(df, d, n):

    for i in range(len(df.columns)):

        k = len(df.iloc[:, i])
        st1 = np.tile(d[1][i], (k, 1))
        st2 = df.iloc[:, i].values.reshape(-1, 1)
        ind = (st1 == st2)
        exc = np.tile(d[0][i], (k, 1)).reshape(k, n, len(d[0][i][0]))[
            np.repeat(ind, n, axis=0).reshape(k, n, len(ind[0]))].reshape(k, n)
        lat_neuron = np.transpose(np.where(exc > 0.1, 1 - exc, np.nan))

        if i == 0:
            lat_neuron_total = lat_neuron
        else:
            lat_neuron_total = np.concatenate((lat_neuron_total, lat_neuron), axis = 0)

    lat_neuron_total[lat_neuron_total == 0] = 0.0001

    return lat_neuron_total

In [7]:
sigm = [0.1, 0.1, 0.2, 0.1]

In [8]:
d = Gaus_neuron(df_, 10, 0.001, sigm)
fin = Lat_Spike(df_, d, 10)

In [9]:
label_dict = {'setosa': 1, 'versicolor': 2, 'virginica': 3}
labels = [label_dict[i] for i in df['species']]
labels = np.array(labels)

In [10]:
fin = np.concatenate((fin, labels.reshape(1,-1)), axis = 0)         # last row is the labels
Final_df = pd.DataFrame(fin)
Final_df

Final_df.to_csv(DATA_PATH + 'data/iris_encoded_mine.csv')

In [11]:
Final_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,0.061244,NaN,NaN,0.594954,NaN,NaN,0.248303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.003942,NaN,0.211232,NaN,NaN,NaN,NaN,0.506744,0.047242,NaN,...,NaN,NaN,NaN,NaN,NaN,0.838971,NaN,NaN,NaN,0.003942
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.181719,NaN,NaN,0.199258,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.286821,NaN
